In [1]:
import numpy as np
from astropy.io import fits
from astropy.coordinates import Angle
from astropy import units as u
from astropy.modeling.models import Gaussian2D as g2d, Gaussian1D as g1d
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
filename = 'etest_cube.fits'
hdul = fits.open(filename)
hdul.info()

Filename: etest_cube.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      78   (512, 512, 65, 1)   float32   
  1  BEAMS         1 BinTableHDU     30   65R x 5C   [1E, 1E, 1E, 1J, 1J]   


In [3]:
hdr = hdul[0].header                        #Header
data = hdul[0].data[0]   

In [17]:
sh = data.shape
t_x,t_y,t_z = sh[1],sh[2],sh[0]

In [18]:
print(t_x,t_y,t_z)

512 512 65


In [6]:
#Calculating Beam to Pixel Ratio
cdel_1 = Angle(hdr['CDELT1'],u.degree)      #pixel increment from header
dmaj = Angle(hdr['BMAJ'],u.degree)          #BMAJ FWHM from header

c_1 = np.abs(cdel_1.arcsec)
d_1= dmaj.arcsec

cdel_2 = Angle(hdr['CDELT2'],u.degree)      #pixel increment from header
dmIN = Angle(hdr['BMIN'],u.degree)          #BMIN FWHM from header

c_2 = np.abs(cdel_1.arcsec)
d_2 = dmIN.arcsec

b_x = int(d_1/c_1);b_y = int(d_2/c_2)

v = hdr['BPA']/70139.80849035224
bpa = Angle(v,u.radian)

In [7]:
r = input('Enter the Number Channels to Span: ')
b_z = int(r)

Enter the Number Channels to Span: 8


In [8]:
n  = input('Enter the Number of Sources: ')
ns  = int(n)

Enter the Number of Sources: 7


In [9]:
a = 10*np.max(data)
Amp = np.random.uniform(a,a/2,ns)
s_x = np.random.randint(0,t_x,ns)
s_y = np.random.randint(0,t_y,ns)
s_z = np.random.randint(0,t_z,ns)

df = pd.DataFrame(list(zip(s_x, s_y, s_z, Amp)),columns =['x_mean', 'y_mean', 'z_mean', 'Amplitude'])
df.to_csv('source_data.csv')
df

,x_mean,y_mean,z_mean,Amplitude
0,40,375,25,0.019546
1,42,247,24,0.019524
2,58,268,55,0.027748
3,20,74,9,0.019165
4,31,113,6,0.019803
5,31,206,27,0.023655
6,36,25,57,0.027572


In [10]:
def Gaussian_Generator(t_x,t_y,t_z,x_m,y_m,z_m,b_x,b_y,b_z,amp,bpa):
    z,y,x = np.ogrid[0:t_z,0:t_y,0:t_x]
    g_2 = g2d(amplitude=amp, x_mean=x_m, y_mean=y_m, x_stddev=b_x, y_stddev=b_y, theta=bpa)
    g_xy = g_2(x,y)
    g_1 = g1d(amplitude=1, mean=z_m, stddev=b_z)
    g_z = g_1(z)
    gs = g_xy*g_z
    return gs

In [11]:
for (x_m, y_m, z_m, amp) in zip(s_x, s_y, s_z, Amp):
    gs = Gaussian_Generator(t_x,t_y,t_z,x_m,y_m,z_m,b_x,b_y,b_z,amp,bpa)
    hdul[0].data[0] = hdul[0].data[0]+gs

ValueError: operands could not be broadcast together with shapes (65,512,512) (65,512,65) 

In [12]:
gs.shape

(65, 512, 65)

In [ ]:
plt.imshow(hdul[0].data[0][45])

In [ ]:
hdul.writeto('new_now_file.fits',overwrite=True)